In [1]:
# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import psutil
import os

In [2]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", 
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
IGNORE = set(["bus", "tvmonitor", "diningtable", "aeroplane", "sofa", "boat", "person", "pottedplant"])

In [17]:
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))
 
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("./MobileNetSSD_deploy.prototxt.txt", "./MobileNetSSD_deploy.caffemodel")
#net = cv2.dnn.readNetFromCaffe("./yolo_deploy.prototxt.txt", "./yolo.caffemodel")
#net = cv2.dnn.readNetFromCaffe("./rcnn.txt", "./bvlc_reference_rcnn_ilsvrc13.caffemodel")
#net = cv2.dnn.readNetFromCaffe("./all_Cnn.prototxt.txt", "./ALL_CNN_C_iter_52000.caffemodel")
  
# initialize the video stream, allow the cammera sensor to warmup,
# and initialize the FPS counter
print("[INFO] starting video stream...")
vs = cv2.VideoCapture("project_video.mp4")
time.sleep(2.0)
fps = FPS().start()

[INFO] loading model...
[INFO] starting video stream...


In [21]:
# loop over the frames from the video stream
grabbed, frame = vs.read()

inputFPS = round(vs.get(cv2.CAP_PROP_FPS), 1)
startTime = time.time()
frameCount = 1
pid = os.getpid()
cpu = []
memory = []
peakFPS = 0

averageFPS =0
fpsList = []
bytesToGB = 1024**3 #Divisor to convert from Gigabytes to bytes
while grabbed:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = imutils.resize(frame, width=540)
    
    
    py = psutil.Process(pid)
    frameStartTime = time.time()
    
    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (720, 540)),
        0.007843, (720, 540), 127.5)
 
    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()
    
    
    currentTrackingFPS = 1.0/(time.time() - frameStartTime)
    fpsList.append(currentTrackingFPS)
    
    #Get resource Metrics
    memory.append(py.memory_info()[0]/bytesToGB)
    cpu.append(psutil.cpu_percent())
    
    
    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]
 
        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.7:
            # extract the index of the class label from the
            # `detections`
            idx = int(detections[0, 0, i, 1])

            # if the predicted class label is in the set of classes
            # we want to ignore then skip the detection
            if CLASSES[idx] in IGNORE:
                continue
    
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
             
            # draw the prediction on the frame
            label = "{}: {:.2f}%".format(CLASSES[idx],
                confidence * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
            
    #Write the fps on the screen
    inputFPSText = "Input FPS: " + str(inputFPS)
    avgTrackingFPSText = "Average Tracking FPS: " + str(round(np.mean(fpsList), 3))
    currentTrackingFPSText = "Current Tracking FPS: " + str(round(currentTrackingFPS, 3))


    #Write on the Frame
    cv2.putText(frame, avgTrackingFPSText, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA )
    cv2.putText(frame, currentTrackingFPSText, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA )
    cv2.putText(frame, inputFPSText, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA )
    
    
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
 
    # update the FPS counter
    fps.update()
    grabbed, frame = vs.read()
 
# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
 
# do a bit of cleanup
cv2.destroyAllWindows()

[INFO] elapsed time: 414.22
[INFO] approx. FPS: 3.02


In [14]:
avgTrackingFPSText
np.max(fpsList)

4.75668711795593